# Data extraction and transformation

### Import Libraries

In [1]:
import os
import re
import csv
import io
import shutil
import pytz
import pandas as pd
import geopandas as gpd
from datetime import datetime
from shapely.geometry import shape, Point
from geopy.distance import distance

### Get the paths to the folders in the dataset

In [2]:
notebook_folder = os.getcwd()
root_project = os.path.abspath(os.path.join(notebook_folder, '.'))
dataset_logs = os.path.abspath(os.path.join(root_project, 'Datos', 'Logs'))
trufi_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App'))
municipios_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos','peru_provincial_simple.geojson'))
csv_file_path = os.path.join(trufi_datos, 'origin-destination.csv')

### Set the time zone of Country

In [3]:
# In this case Arequipa, Peru
analysis_timezone = pytz.timezone('America/Lima')

### Extract logs requests to origin-destination.csv

In [4]:
def extract_route_info(log_line):
    # Regular expression to extract specific information from route requests
    route_pattern_with_id = re.compile(r'GET /otp/routers/default/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+).*?Trufi/.*?/([a-f0-9-]+)')

    match_with_id = route_pattern_with_id.search(log_line)
    # Initialize variables with default values
    origin_latitude = origin_longitude = dest_latitude = dest_longitude = id_user = None
    
    try:
        if match_with_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = match_with_id.groups()
        return origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user
    except Exception as e:
        print(f"Error processing line: {log_line}")
        print(f"Error: {e}")

    return None

def process_log_file(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if 'GET /otp/routers/default/plan' in line:
                date_str = re.search(r'\[([^:]+:[^ ]+)', line).group(1)
                # Convert the date to a datetime object and add the La Paz timezone
                date_time = datetime.strptime(date_str, '%d/%b/%Y:%H:%M:%S').replace(tzinfo=pytz.utc).astimezone(analysis_timezone)
                route_info = extract_route_info(line)
                if route_info:
                    origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = route_info
                    yield [date_time.strftime('%Y-%m-%d %H:%M:%S'), origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user]
                    
# Get the list of files and sort them by numeric prefix
file_pattern = re.compile(r'^\d{2}-')
files = os.listdir(dataset_logs)
log_files = [file for file in files if file_pattern.match(file)]
log_files.sort(key=lambda x: int(x.split('-')[0]))

header = ['date', 'origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude', 'userID']

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)
    for log_file in log_files:
        file_path = os.path.join(dataset_logs, log_file)
        
        route_info_generator = process_log_file(file_path)
        
        # Write the lines to the CSV file
        csv_writer.writerows(route_info_generator)

print(f"Routes matching the pattern have been saved to {csv_file_path}")

Routes matching the pattern have been saved to D:\TrufiData\Datos\Registros de Trufi App\origin-destination.csv


### Discard requests without user ID 

In [5]:
def filter_requests_with_userid(csv_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path, parse_dates=['date'])

    # Count rows before filtering
    total_rows_before = len(df)

    # Filter the requests that have userID
    df_filtered = df[df['userID'].notnull()]

    # Count rows after filtering
    total_rows_after = len(df_filtered)

    # Calculate how many rows were removed
    rows_removed = total_rows_before - total_rows_after

    # Overwrite the original file with the filtered requests
    df_filtered.to_csv(csv_file_path, index=False)

    return total_rows_before, rows_removed

# Call the function to filter the requests
total_rows_before, rows_removed = filter_requests_with_userid(csv_file_path)

# Print the number of removed rows
print(f"Removed {rows_removed} rows without userID from {total_rows_before}.")

Removed 1441 rows without userID from 23892.


### Number of users to within the evaluated date range

In [6]:
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Get the number of unique users
unique_users_count = df['userID'].nunique()

# Print the number of unique users
print(f"The number of unique users is: {unique_users_count}")

The number of unique users is: 4408


### Generate syntetic data

In [7]:
# Read the CSV file with time and distance features
df = pd.read_csv(csv_file_path, parse_dates=['date'], encoding='latin1')

# Add time and distance features
df['hour'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.dayofweek  # Monday: 0, Sunday: 6
df['day_of_month'] = df['date'].dt.day
df['weekend'] = (df['date'].dt.weekday >= 5).astype(int)  # 1 if weekend, 0 if not
# Add variable for the week number of the year
df['year_week_number'] = df['date'].dt.strftime('%Y%U')

# Function to assign the time of day period
def assign_time_of_day(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'night'

# Apply the function to create the new variable 'time_of_day'
df['time_of_day'] = df['hour'].apply(assign_time_of_day)

# Show the resulting DataFrame with the new variable
print(df)

# Save the filtered DataFrame with the new variable to a new CSV file
df.to_csv(csv_file_path, index=False)

print(f"The filtered requests have been saved to: {csv_file_path}")


                     date  origin_latitude  origin_longitude  \
0     2024-09-03 23:28:18       -16.375103        -71.557241   
1     2024-09-03 23:28:38       -16.429629        -71.529506   
2     2024-09-03 23:29:13       -16.375103        -71.557241   
3     2024-09-03 23:29:21       -16.429629        -71.529506   
4     2024-09-03 23:30:52       -16.375103        -71.557241   
...                   ...              ...               ...   
22446 2024-09-30 20:20:59       -16.436882        -71.530523   
22447 2024-09-30 20:23:45       -16.452287        -71.525863   
22448 2024-09-30 20:25:41       -16.420176        -71.500390   
22449 2024-09-30 20:25:57       -16.420176        -71.500390   
22450 2024-09-30 20:32:33       -16.407343        -71.539034   

       destination_latitude  destination_longitude  \
0                -16.429629             -71.529506   
1                -16.375103             -71.557241   
2                -16.429629             -71.529506   
3              

### Exclude requests for routes with a distance of less than 300 meters between origin and destination.

In [8]:
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Create points for the latitude and longitude columns
df['origin_point'] = df.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
df['destination_point'] = df.apply(lambda row: Point(row['destination_longitude'], row['destination_latitude']), axis=1)

# Calculate the distance between origin and destination in meters
df['distance'] = df.apply(lambda row: distance((row['origin_latitude'], row['origin_longitude']),
                                               (row['destination_latitude'], row['destination_longitude'])).meters, axis=1)

# Filter the DataFrame to keep only the points that have a distance greater than 300 meters
df_filtered = df[df['distance'] > 300]

In [9]:
# Count rows before filtering
rows_before = len(df)

# Count rows after filtering
rows_after = len(df_filtered)

# Print the number of rows before and after filtering
print(f"Requests before filtering: {rows_before}")
print(f"Requests after filtering: {rows_after}")

# Save the filtered DataFrame to a new CSV file without the origin_point and destination_point columns
df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)
df_filtered.to_csv(csv_file_path, index=False)

print(f"The filtered requests have been saved to: {csv_file_path}")

Requests before filtering: 22451
Requests after filtering: 21026


C:\Users\idaas\AppData\Local\Temp\ipykernel_16272\3715189283.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)


The filtered requests have been saved to: D:\TrufiData\Datos\Registros de Trufi App\origin-destination.csv


### Generate variables for origin and destination municipalities

In [10]:
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Add the 'origin_municipio' and 'dest_municipio' columns with default values
df['origin_municipio'] = 'external'
df['dest_municipio'] = 'external'

# Save the updated DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)

In [11]:
# Load the GeoDataFrame from the GeoJSON
arequipa_gdf = gpd.read_file(municipios_datos)

In [12]:
# Update the get_city_from_coords function to return NOMBPROV
def get_city_from_coords(coords):
    point = Point(coords)
    # Iterate over the features of the filtered Arequipa GeoJSON
    for index, row in arequipa_gdf.iterrows():
        city_geometry = row['geometry']
        # Check if the point is within the city's geometry
        if point.within(city_geometry):
            return row['NOMBPROV']  # Return the NOMBPROV instead of FIRST_NOMB
    
    return 'external'

# Add the columns 'origin_municipio' and 'dest_municipio' to the DataFrame
columns_to_add = ['origin_municipio', 'dest_municipio']

# Create an empty DataFrame with the new columns
df = pd.DataFrame(columns=columns_to_add)
# Open the CSV file for reading and read the existing columns
with open(csv_file_path, 'r') as csvfile:
    # Read the first line to get the column names
    existing_columns = csvfile.readline().strip().split(',')

    # Create an empty DataFrame
    df = pd.DataFrame(columns=existing_columns)

    # Create a temporary file to write the updated lines
    with open('temp_csvfile.csv', 'w', newline='') as temp_csvfile:
        # Write the column names to the temporary file
        temp_csvfile.write(','.join(existing_columns) + '\n')

        for i, line in enumerate(csvfile):
            # Read a line from the CSV file
            row = pd.read_csv(io.StringIO(line), header=None, names=existing_columns).iloc[0]

            # Get municipality names for origin and destination
            origin_municipio = get_city_from_coords((row['origin_longitude'], row['origin_latitude']))
            dest_municipio = get_city_from_coords((row['destination_longitude'], row['destination_latitude']))

            # Add the values to the DataFrame
            row['origin_municipio'] = origin_municipio
            row['dest_municipio'] = dest_municipio

            # Write the updated line to the temporary file
            temp_csvfile.write(','.join(map(str, row.values)) + '\n')
shutil.move('temp_csvfile.csv', csv_file_path)


'D:\\TrufiData\\Datos\\Registros de Trufi App\\origin-destination.csv'